# 01 - Matches par équipe (fetch/debug)
Scrape les matches via l'API OpenDota `/teams/{team_id}/matches`, sauvegarde brut, et aperçu rapide pour debug.
- Clé via `OPENDOTA_KEY` (.env).
- Helpers réutilisables dans `src.dota_data.api`.


In [1]:
from pathlib import Path
import sys

def _find_root():
    candidates = [Path.cwd()] + list(Path.cwd().parents[:4])
    for cand in candidates:
        if (cand / "src").exists():
            return cand
    return Path.cwd()

ROOT = _find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
print(f"Using project root: {ROOT}")


Using project root: /home/ju/Documents/Dev/Dota-Datas


In [2]:
from pathlib import Path
import json
import pandas as pd
from dotenv import load_dotenv

from src.dota_data.api import (
    load_api_key,
    build_session,
    load_team_list,
    fetch_team_matches,
    annotate_matches_with_team,
    write_json,
)

load_dotenv()
teams = load_team_list(ROOT / "data/teams_to_look.csv")
api_key = load_api_key(load_env_file=False)
session = build_session(api_key)
print(f"Teams chargées: {len(teams)}")


Teams chargées: 173


In [3]:
team_matches = []
for team in teams:
    matches = fetch_team_matches(team["TeamID"], session=session)
    team_matches.append({"team": team, "matches": matches})

total_matches = sum(len(t["matches"] ) for t in team_matches)
print(f"Matches récupérés: {total_matches} sur {len(teams)} équipes")


Matches récupérés: 48148 sur 173 équipes


In [4]:
flat_rows = []
for entry in team_matches:
    flat_rows.extend(annotate_matches_with_team(entry["matches"], entry["team"]["TeamID"], entry["team"]["TeamName"]))

matches_df = pd.DataFrame(flat_rows)
if not matches_df.empty:
    matches_df["start_dt"] = pd.to_datetime(matches_df["start_time"], unit="s")
matches_df.head()


,match_id,radiant_win,radiant_score,dire_score,radiant,duration,start_time,leagueid,league_name,cluster,opposing_team_id,opposing_team_name,opposing_team_logo,_source_team_id,_source_team_name,start_dt
0,8572758153,True,42,22,False,2328,1763897576,18920,PGL Wallachia 2025 Season 6,272,9338413,MOUZ,https://cdn.steamusercontent.com/ugc/149367842...,2163,Team Liquid,2025-11-23 11:32:56
1,8572646777,False,23,29,False,2909,1763892992,18920,PGL Wallachia 2025 Season 6,271,9338413,MOUZ,https://cdn.steamusercontent.com/ugc/149367842...,2163,Team Liquid,2025-11-23 10:16:32
2,8572546517,False,17,44,True,2326,1763888685,18920,PGL Wallachia 2025 Season 6,272,9338413,MOUZ,https://cdn.steamusercontent.com/ugc/149367842...,2163,Team Liquid,2025-11-23 09:04:45
3,8571364449,False,21,28,True,2488,1763826709,18920,PGL Wallachia 2025 Season 6,271,7119388,Team Spirit,https://cdn.steamusercontent.com/ugc/183917912...,2163,Team Liquid,2025-11-22 15:51:49
4,8571247569,False,11,19,True,1753,1763823160,18920,PGL Wallachia 2025 Season 6,274,7119388,Team Spirit,https://cdn.steamusercontent.com/ugc/183917912...,2163,Team Liquid,2025-11-22 14:52:40


In [5]:
out_path = Path("data/interim/team_matches_raw.json")
write_json(team_matches, out_path)
print(f"Sauvegardé {out_path} ({out_path.stat().st_size/1024:.1f} KB)")


Sauvegardé data/interim/team_matches_raw.json (25029.6 KB)


In [6]:
# Debug rapide: volume par équipe et stats de base
if not matches_df.empty:
    display(matches_df.groupby("_source_team_name")["match_id"].nunique().sort_values(ascending=False))
    display(matches_df.describe(include="all"))


_source_team_name
Team Liquid          2898
Shopify Rebellion    2674
Natus Vincere        2553
Ascent Esports       2473
Vici Gaming          2473
                     ... 
Fate's Edge             2
TheSleepers             2
MOPSYARKI               1
YYGQQ                   1
tubao                   1
Name: match_id, Length: 156, dtype: int64

,match_id,radiant_win,radiant_score,dire_score,radiant,duration,start_time,leagueid,league_name,cluster,opposing_team_id,opposing_team_name,opposing_team_logo,_source_team_id,_source_team_name,start_dt
count,4.814800e+04,48148,48148.000000,48148.000000,48148,48148.000000,4.814800e+04,48148.000000,48148,48148.000000,4.814800e+04,48148,47199,4.814800e+04,48148,48148
unique,NaN,2,NaN,NaN,2,NaN,NaN,NaN,767,NaN,NaN,1541,1482,NaN,156,NaN
top,NaN,True,NaN,NaN,True,NaN,NaN,NaN,European Pro League 2024-2025 Season,NaN,NaN,Team Liquid,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,NaN,Team Liquid,NaN
freq,NaN,24405,NaN,NaN,24106,NaN,NaN,NaN,2070,NaN,NaN,1366,1366,NaN,2898,NaN
mean,6.382074e+09,NaN,22.531195,22.420225,NaN,2275.224745,1.651139e+09,13916.392560,NaN,208.452978,6.254953e+06,NaN,NaN,5.712155e+06,NaN,2022-04-28 09:37:41.803958528
min,2.227015e+07,NaN,0.000000,0.000000,NaN,364.000000,1.340567e+09,4.000000,NaN,1.000000,2.000000e+00,NaN,NaN,3.600000e+01,NaN,2012-06-24 19:43:27
25%,5.458070e+09,NaN,13.000000,12.000000,NaN,1834.000000,1.591499e+09,12027.000000,NaN,152.000000,2.108395e+06,NaN,NaN,7.262280e+05,NaN,2020-06-07 02:58:48.750000128
50%,7.340925e+09,NaN,23.000000,23.000000,NaN,2199.000000,1.695043e+09,15728.000000,NaN,192.000000,8.255888e+06,NaN,NaN,7.732977e+06,NaN,2023-09-18 13:19:59.500000
75%,8.173318e+09,NaN,32.000000,32.000000,NaN,2629.000000,1.739538e+09,17527.000000,NaN,272.000000,9.256405e+06,NaN,NaN,9.247798e+06,NaN,2025-02-14 13:02:21.249999872
max,8.597442e+09,NaN,76.000000,80.000000,NaN,8199.000000,1.765289e+09,65013.000000,NaN,412.000000,9.981139e+06,NaN,NaN,9.980558e+06,NaN,2025-12-09 14:03:42
